### Some Cleaning steps

In [1]:
import pandas as pd
import numpy as np
import re
import recommender as r
import matplotlib.pyplot as plt
import seaborn as sns
# NLP
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy
nlp = spacy.load('en_core_web_sm')

%matplotlib inline

df = pd.read_csv('user-item-interactions.csv')
df_content = pd.read_csv('articles_community.csv')
del df['Unnamed: 0']
del df_content['Unnamed: 0']

# make sure all articles from df are in df_content
df = pd.merge(df, df_content[['article_id']], on='article_id', how='inner')

df_content = df_content.drop(labels='doc_status', axis=1)
df_content = df_content.drop_duplicates()

#df = df.replace('no_email', np.nan)

def email_mapper():
    coded_dict = dict()
    cter = 1
    email_encoded = []
    
    for val in df['email']:
        if val not in coded_dict:
            coded_dict[val] = cter
            cter+=1
        
        email_encoded.append(coded_dict[val])
    return email_encoded

email_encoded = email_mapper()
del df['email']
df['user_id'] = email_encoded

# Rename my article name field to be the same in both dfs
df_content['title'] = df_content['doc_full_name']
df_content = df_content.drop(labels='doc_full_name', axis=1)

df.article_id = df.article_id.astype('int64')
df_content.article_id = df_content.article_id.astype('int64')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/medhyvinceslas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/medhyvinceslas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/medhyvinceslas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df.head()

,article_id,title,user_id
0,593,upload files to ibm data science experience us...,1
1,593,upload files to ibm data science experience us...,2
2,593,upload files to ibm data science experience us...,3
3,593,upload files to ibm data science experience us...,4
4,593,upload files to ibm data science experience us...,3


In [3]:
df_content.head()

,doc_body,doc_description,article_id,title
0,Skip navigation Sign in SearchLoading...\r\n\r...,Detect bad readings in real time using Python ...,0,Detect Malfunctioning IoT Sensors with Streami...
1,No Free Hunch Navigation * kaggle.com\r\n\r\n ...,"See the forest, see the trees. Here lies the c...",1,Communicating data science: A guide to present...
2,☰ * Login\r\n * Sign Up\r\n\r\n * Learning Pat...,Here’s this week’s news in Data Science and Bi...,2,"This Week in Data Science (April 18, 2017)"
3,"DATALAYER: HIGH THROUGHPUT, LOW LATENCY AT SCA...",Learn how distributed DBs solve the problem of...,3,DataLayer Conference: Boost the performance of...
4,Skip navigation Sign in SearchLoading...\r\n\r...,This video demonstrates the power of IBM DataS...,4,Analyze NY Restaurant data using Spark in DSX


In [4]:
# number of times the user seen the article
pre_data = dict(df.groupby(['user_id', 'article_id'])['article_id'].count())

list_user = []
list_article_id = []
list_nb_interactions = []

for key, val in pre_data.items():
    list_user.append(key[0])
    list_article_id.append(key[1])
    list_nb_interactions.append(val)
    
zipped_list = list(zip(list_user, list_article_id, list_nb_interactions))
interaction_user = pd.DataFrame(zipped_list, columns=['user_id','article_id','nb_interactions_user_article'])

df = pd.merge(df, interaction_user, on=['user_id','article_id'])

# The nb interactions was shown implicitly with the number of rows
# Now I have information about the number of interactions by creating this column
# I can drop duplicated rows
df = df.drop_duplicates(keep='first')
df = df.reset_index().drop(labels='index', axis=1)
df.head()

,article_id,title,user_id,nb_interactions_user_article
0,593,upload files to ibm data science experience us...,1,1
1,593,upload files to ibm data science experience us...,2,1
2,593,upload files to ibm data science experience us...,3,2
3,593,upload files to ibm data science experience us...,4,1
4,593,upload files to ibm data science experience us...,5,1


I don't have any date of the interraction between user and article, so just for the module, I'll simulate a date field set to 0. Popularity of an item is computed by the weighted rating calcul shown in recommender_function.py, but in case of tie, the more recent date will be first, so by putting a same value for all the records, I'm not influence the importance of an article. Here I chose 0

In [5]:
df['date'] = 0

##### Prepare TFIDF Matrice for Content Based Recommendations

In [6]:
def tokenize(text):
    """
    This function do the following steps:
    - Normalize, remove ponctuations
    - Tokenize the input (str)
    - Remove english stopwords
    - Lemmatize but keep the gramatical sense of the word
    - Clean White spaces
    """
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words("english")]
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    words_clean = [word.strip() for word in words]
    return words_clean

In [7]:
tfidf = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2))
df_content['doc_description'] = df_content['doc_description'].fillna('')
tfidf_matrix = tfidf.fit_transform(df_content['doc_description'])

In [8]:
tfidf_matrix.shape

(1056, 17507)

****
****

#### Recommendations

In [9]:
rec = r.Recommender(df_items=df_content,                            # df that contains all the items with description and more
                    df_reviews=df,                                  # df that contains interactions between users and items
                    #based_similarity_col='doc_description',         # The column name that you want to based on to compute similarity between items
                    item_name_colname='title',                      # The title column of the df (this can be use with the 1st df or the 2nd, that why I wanted the same name for both)
                    user_id_colname='user_id',                      # The name of the user id column
                    item_id_colname='article_id',                   # The name of the item id column
                    rating_col_name='nb_interactions_user_article', # The rating column
                    date_col_name='date')                           # The date column

In [10]:
rec.fit(iters=10, latent_features=10)

Create User-Item matrix...
Train data with Funk Sigular Value Decomposition...
Iterations 		 Mean Squared Error 
	1 		 2.4964122936500592 
	2 		 2.3207887788247876 
	3 		 2.1679583890310745 
	4 		 2.034084338859403 
	5 		 1.9161039259364883 
	6 		 1.811549855757542 
	7 		 1.718417371258855 
	8 		 1.6350643805981449 
	9 		 1.5601356397025377 
	10 		 1.4925046604766596 


#### investigate the user-Item matrix created

In [11]:
rec.user_item_df.head()

article_id,0,2,4,8,9,12,14,15,16,18,...,1028,1030,1035,1038,1042,1043,1044,1047,1048,1050
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Get some Infos

In [12]:
def info():
    user_item = rec.user_item_df
    nb_user = rec.n_users
    nb_items = rec.n_items
    item_name = rec.item_name_colname
    item_id = rec.item_id_colname
    u_mat = rec.user_mat
    i_mat = rec.item_mat
    user_high_rate = list(dict(user_item.mean(axis=1).sort_values(ascending=False).head(1)).keys())[0]
    movie_id_high_rate = list(dict(user_item.mean(axis=0).sort_values(ascending=False).head(1)).keys())[0]
    movie_name_high_rate = tuple(df_content[df_content[item_id] == movie_id_high_rate][item_name])[0]
    

    print(f"Nb of users: {nb_user}")
    print(f"Nb of items: {nb_items}")
    print(f"The user_id with the highest avg rating given: {user_high_rate}")
    print(f"The article_id with the highest avg rating received: {movie_id_high_rate}")
    print(f"The article name with the highest avg rating received: {movie_name_high_rate}")
    print(f"Shape of the U matrix: {u_mat.shape}")
    print(f"Shape of the V(transpose) matrix: {i_mat.shape}")

info()

Nb of users: 4258
Nb of items: 437
The user_id with the highest avg rating given: 1102
The article_id with the highest avg rating received: 50
The article name with the highest avg rating received: Graph-based machine learning
Shape of the U matrix: (4258, 10)
Shape of the V(transpose) matrix: (10, 437)


****
##### To make recommendation: given a user id we want to find similar users to this user in order to recommend items. Similarity are found by computing the dot product of characteristics of the user with its transpose, the more the result of an user-user pair is high, the more they have in common.

In [13]:
df_user_similarity = rec.user_item_df.reset_index().replace(np.nan, 0)
def prep_get_similar_user():
    user_content = np.array(df_user_similarity.iloc[:,1:])
    user_content_transpose = np.transpose(user_content)
    dot_prod = user_content.dot(user_content_transpose)
    return dot_prod

dot_product_matrix_user = prep_get_similar_user()

##### Prepare a function to display results

In [15]:
def display_recommendations(rec_ids, rec_names, message, rec_ids_users, rec_user_articles):
    
    if type(rec_ids) == type(None):
        print(f"{message}")
    
    else:
        dict_id_name = dict(zip(rec_ids, rec_names))
        
        if type(rec_ids_users) != type(None):
            print('Matrix Factorisation SVD:')
            print(f"\t{message}")
            
            for key, val  in dict_id_name.items():
                print(f"\t- ID items: {key}")
                print(f"\tName: {val}\n")

            print('CF User Based:')
            print('\tUser that are similar to you also seen:\n')
            for i in rec_user_articles[:5]:
                print(f"\t- {i}")
        else:
            print(f"\t{message}")
            dict_id_name = dict(zip(rec_ids, rec_names))
            for key, val  in dict_id_name.items():
                print(f"\t- ID items: {key}")
                print(f"\tName: {val}\n")

***
#### Existing user

- Because it is an existing user, recommendations are made using FunkSVD (matrix factorisation), it will predict the rating it will give to all items and get back the items associate with the the top predicted rate. The dot_product_matrix will not be used but is requiered in case of you want to find a similar item to another instead of finding best item for a user. 

In [16]:
rec_ids, rec_names, message, rec_ids_users, rec_user_articles = rec.make_recommendations(_id=3,
                                                                                         dot_prod_user= dot_product_matrix_user,
                                                                                         tfidf_matrix=tfidf_matrix,
                                                                                         _id_type='user',
                                                                                         rec_num=5)
display_recommendations(rec_ids, rec_names, message, rec_ids_users, rec_user_articles)

Matrix Factorisation SVD:
	Glad to see you again! recommended for you:

	- ID items: 973
	Name: Recent trends in recommender systems

	- ID items: 984
	Name: The Data Processing Inequality

	- ID items: 346
	Name: Fighting Gerrymandering: Using data science to draw fairer congressional districts

	- ID items: 195
	Name: Artificial Intelligence, Ethically Speaking – Inside Machine learning – Medium

	- ID items: 626
	Name: Analyze Db2 Warehouse on Cloud data in RStudio in DSX

CF User Based:
	User that are similar to you also seen:

	- upload files to ibm data science experience using the command line


***
#### New User

- Because it is new user, recommendations are given using ranked based method, which simply return back the most popular items according to the ratings given by users, the number of ratings, the recency of the ratings. The dot_product_matrix will not be used but is requiered in case of you want to find a similar item to another instead of finding best item for a user.

In [17]:
rec_ids, rec_names, message, rec_ids_users, rec_user_articles = rec.make_recommendations(_id=8000,
                                                                                         dot_prod_user= dot_product_matrix_user,
                                                                                         tfidf_matrix=tfidf_matrix,
                                                                                         _id_type='user',
                                                                                         rec_num=5)

print('Ranked Based:')
display_recommendations(rec_ids, rec_names, message, rec_ids_users, rec_user_articles)

Ranked Based:
	Hey, you are new here, this is for you:

	- ID items: 221
	Name: How smart catalogs can turn the big data flood into an ocean of opportunity

	- ID items: 50
	Name: Graph-based machine learning

	- ID items: 232
	Name: Self-service data preparation with IBM Data Refinery

	- ID items: 43
	Name: Deep Learning With Tensorflow Course by Big Data University

	- ID items: 732
	Name: Rapidly build Machine Learning flows with DSX



****
#### Existing items

- Here we enter an item id and would like to find similar items using the dot_product_matrix computed earlier.

In [19]:
rec_ids, rec_names, message, rec_ids_users, rec_user_articles = rec.make_recommendations(_id=593,
                                                                                         dot_prod_user= dot_product_matrix_user,
                                                                                         tfidf_matrix=tfidf_matrix,
                                                                                         _id_type='item',
                                                                                         rec_num=5)
print('Content Based:')
display_recommendations(rec_ids, rec_names, message, rec_ids_users, rec_user_articles)

Content Based:
	Similar items for id:593, corresponding to Upload Files to IBM Data Science Experience Using the Command Line:

	- ID items: 593
	Name: Upload Files to IBM Data Science Experience Using the Command Line

	- ID items: 600
	Name: Access IBM Analytics for Apache Spark from RStudio

	- ID items: 809
	Name: Use the Machine Learning Library

	- ID items: 161
	Name: Use the Machine Learning Library in Spark

	- ID items: 893
	Name: Use the Machine Learning Library in IBM Analytics for Apache Spark



#### Simulate an error by passing a non existing item id

In [20]:
rec_ids, rec_names, message, rec_ids_users, rec_user_articles = rec.make_recommendations(_id=1087630,
                                                                                         dot_prod_user= dot_product_matrix_user,
                                                                                         tfidf_matrix=tfidf_matrix,
                                                                                         _id_type='item',
                                                                                         rec_num=5)


display_recommendations(rec_ids, rec_names, message, rec_ids_users, rec_user_articles)

We can't make recommendation for this item, please makesure the data was updated with this item.



##### Imagine that we have an article that we want to promote, but we want the top 10 users who may interested by this offer, the item id is 984.

In [21]:
user_item = rec.user_item_df
def may_interested_by(item_id, top_n=10):
    pred = {}
    # iterate over each users and predict the rate it will give to this movie
    for user in user_item.index:
        pred[user] = rec.predict_rating(user_id=user, item_id=item_id)

    top_10_pairs = sorted(pred.items(), key=lambda x: x[1], reverse=True)[:top_n]
    top_10_user_ids = []

    for i in top_10_pairs:
        top_10_user_ids.append(i[0])

    return top_10_user_ids

print('These users may interested by this item:')
for i in may_interested_by(984, 10):
    print(f"- {i}")

These users may interested by this item:
- 3951
- 2759
- 1549
- 3445
- 2457
- 927
- 3580
- 2489
- 3161
- 3947
